In [1]:
# Map Libraries
from branca.element import Figure
import folium as fl
from folium.plugins import MarkerCluster, FeatureGroupSubGroup, GroupedLayerControl

# Data Managment
import sqlite3
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

Create connection to Database

In [2]:
con = sqlite3.connect('../database.db')
cur = con.cursor()

get tables as df

In [3]:
query = """
SELECT 
    businessType,
    url,
    propertyType,
    builtArea,
    area,
    bathroomsNumber,
    roomsNumber,
    parkingNumber,
    price,
    stratum,
    lat,
    lon
FROM
    Properties
"""
Properties = pd.read_sql_query(query, con)
geometry = [Point(lon, lat) for lon, lat in zip(Properties['lon'], Properties['lat'])]
geoProperties = gpd.GeoDataFrame(Properties, geometry=geometry)
print(geoProperties.shape)
geoProperties.head()

(8265, 13)


,businessType,url,propertyType,builtArea,area,bathroomsNumber,roomsNumber,parkingNumber,price,stratum,lat,lon,geometry
0,Arriendo,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento,28.00,28.00,1.0,1.0,1.0,1600000.0,4,4.653850,-74.056250,POINT (-74.05625 4.65385)
1,Arriendo,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento,233.00,235.00,3.0,3.0,2.0,9000000.0,6,4.667340,-74.049110,POINT (-74.04911 4.66734)
2,Venta,https://www.metrocuadrado.com/inmueble/venta-l...,Lote o Casalote,188.00,187.50,NaN,NaN,NaN,360000000.0,2,4.563945,-74.132912,POINT (-74.13291 4.56394)
3,Venta,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento,74.72,74.72,3.0,3.0,2.0,380000000.0,4,4.744681,-74.032646,POINT (-74.03265 4.74468)
4,Venta,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento,66.61,66.61,2.0,2.0,2.0,470000000.0,5,4.694066,-74.060853,POINT (-74.06085 4.69407)


In [71]:
geoProperties['businessType'].value_counts()

businessType
Venta               5510
Arriendo            2702
Venta y arriendo      53
Name: count, dtype: int64

In [72]:
geoProperties['propertyType'].value_counts()

propertyType
Apartamento                 5017
Casa                        1279
Oficina                      886
Local Comercial              489
Bodega                       368
Lote o Casalote               84
Consultorio                   67
Edificio de Apartamentos      49
Edificio de Oficinas          26
Name: count, dtype: int64

In [39]:
def html_creator(data):
    bathrooms = data.bathroomsNumber
    rooms = data.roomsNumber
    parking = data.parkingNumber

    if data.bathroomsNumber < 1:
        bathrooms = 'N/A'
    if data.roomsNumber < 1:
        rooms = 'N/A'
    if data.parkingNumber < 1:
        parking = 'N/A'
    html_str = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Property Information</title>
        <style>
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}

            th, td {{
                border: 1px solid #ddd;
                padding: 12px;
                text-align: left;
            }}

            th {{
                background-color: #f2f2f2;
            }}

            td.url {{
                background-color: #e6f7ff; /* Light blue for URLs */
            }}
        </style>
    </head>
    <body>

    <!-- Create a table to organize property information -->
    <table>
        <tr>
            <th>Attribute</th>
            <th>Value</th>
        </tr>
        <tr>
            <td>Precio para</td>
            <td>{str(data.businessType)}</td>
        </tr>
        <tr>
            <td>Precio</td>
            <td>$ {data.price:,}</td>
        </tr>
        <tr>
            <td>Area</td>
            <td>{str(data.area)} mt2</td>
        </tr>
        <tr>
            <td>Estrato</td>
            <td>{str(data.stratum)}</td>
        </tr>
        <tr>
            <td>Numero de alcobas</td>
            <td>{str(rooms)}</td>
        </tr>
        <tr>
            <td>Numero de baños</td>
            <td>{str(bathrooms)}</td>
        </tr>
        <tr>
            <td>Numero de Parqueaderos</td>
            <td>{str(parking)}</td>
        </tr>
        <tr>
            <td>Tipo de Propiedad</td>
            <td>{str(data.propertyType)}</td>
        </tr>
        <tr>
            <td class="url">URL</td>
            <td class="url"><a href="{str(data.url)}">Enlace</a></td>
        </tr>
    </table>

    </body>
    </html>
    """
    return html_str

legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: auto; z-index:9999; font-size:14px;">
<i class="fa fa-home fa-2x" style="color:blue"></i> - Apartamento<br>
<i class="fa fa-home fa-2x" style="color:green"></i> - Casa<br>
<i class="fa fa-home fa-2x" style="color:red"></i> - Oficina<br>
<i class="fa fa-home fa-2x" style="color:purple"></i> - Bodega<br>
<i class="fa fa-home fa-2x" style="color:orange"></i> - Local Comercial<br>
<i class="fa fa-home fa-2x" style="color:gray"></i> - Lote o Casalote<br>
<i class="fa fa-home fa-2x" style="color:cyan"></i> - Edificio de Apartamentos<br>
<i class="fa fa-home fa-2x" style="color:pink"></i> - Consultorio<br>
<i class="fa fa-home fa-2x" style="color:brown"></i> - Edificio de Oficinas<br>
</div>
"""

propertyType_mapping = {
    'Apartamento': 'blue',
    'Casa': 'green',
    'Oficina': 'red',
    'Bodega': 'purple',
    'Local Comercial': 'orange',
    'Lote o Casalote': 'gray',
    'Edificio de Apartamentos': 'cadetblue',
    'Consultorio': 'pink',
    'Edificio de Oficinas': 'beige'
}

businessType_list = geoProperties['businessType'].unique().tolist()
propertyType_list = geoProperties['propertyType'].unique().tolist()

In [80]:
min_lon, max_lon =  -74.2508, -74.0041
min_lat, max_lat = 4.4697, 4.8529
bog_location = (4.6479362,-74.0868797)

m = fl.Map(
    location = (4.6479362,-74.0868797),
    zoom_start=11,
    control_scale=True,
    zoom_control=False,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon)

cluster = MarkerCluster(control=False).add_to(m)

layers = {}
sublayers = {}
layers_2= {}
for B in businessType_list:
    layers[B] = FeatureGroupSubGroup(cluster, name = B).add_to(m)
    for P in propertyType_list:
        sublayers[f"{B}_{P}"] = FeatureGroupSubGroup(layers[B], name = P).add_to(m)

for P in propertyType_list:
    layers_2[P] = FeatureGroupSubGroup(cluster, name = B).add_to(m)

for i in tqdm(geoProperties.itertuples()):
    # Get custom settings
    iframe_html = html_creator(i)

    # Add aesthetics to marker
    icon = fl.Icon(icon='house', color = propertyType_mapping[i.propertyType])
    iframe = fl.IFrame(html = iframe_html, width=400, height=600)
    popup = fl.Popup(iframe)
    P = i.propertyType
    B = i.businessType
    
    marker = fl.Marker(location=(i.lat, i.lon),
                icon=icon, popup=fl.Popup(iframe)).add_to(sublayers[f"{B}_{P}"])

for B in businessType_list:
    groups = {
        B: [value for k,value in sublayers.items() if f'{B}_' in k]
    }

    GroupedLayerControl(
        groups=groups,
        exclusive_groups=True,
        collapsed=True,
        position = 'topright'
    ).add_to(m)

GroupedLayerControl(
        groups={'business': list(layers.values())},
        exclusive_groups=True,
        collapsed=True,
        position = 'topleft'
    ).add_to(m)

m.get_root().html.add_child(fl.Element(legend_html))

m.save('map.html')

0it [00:00, ?it/s]

8265it [00:03, 2101.82it/s]
